In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 508.9 kB/s eta 0:00:00


In [ ]:
from google.colab import files
from google.colab import drive
from google.colab import userdata
import pandas as pd
import csv

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
output_file = "/content/drive/My Drive/sum_test.csv"

df = pd.read_csv(output_file)
df.head()

,bug_report,ground_truth_summary
0,Build ID: 20090619-0625 Steps To Reproduce: 0...,[ui] IU Properties dialog improvements
1,The Equinox Resources page (see url) has outda...,Outdated links on website
2,When the hierarchy view has no corresponding s...,"TVT: The ""empty"" Hierarchy view message is con..."
3,"In the 1.0 version of Eclipse, when you place ...",Bracket matching should highlight both brackets
4,Build 20021216 The Java->JUnit preference pag...,[JUnit] Preference page: stack filter should b...


In [ ]:
df['bug.id'] = df.index
df.head()

,bug_report,ground_truth_summary,bug.id
0,Build ID: 20090619-0625 Steps To Reproduce: 0...,[ui] IU Properties dialog improvements,0
1,The Equinox Resources page (see url) has outda...,Outdated links on website,1
2,When the hierarchy view has no corresponding s...,"TVT: The ""empty"" Hierarchy view message is con...",2
3,"In the 1.0 version of Eclipse, when you place ...",Bracket matching should highlight both brackets,3
4,Build 20021216 The Java->JUnit preference pag...,[JUnit] Preference page: stack filter should b...,4


In [ ]:
output_file2 = "/content/drive/My Drive/gpt3.5_eval_sumllama.csv"


df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Mozilla blocks and crashes when using invalid ...
1,26540,Dark theme makes DPR difficult to see/click in...
2,4320,Optimization bug in SingleNameReference affect...
3,14545,Insecure password warning not showing with dis...
4,23198,Connection errors with IRC server using proxy.


In [ ]:
output_file2 = "/content/drive/My Drive/llama3-br-ft-eval-sumllama.csv"


df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Mozilla crashes when installing a theme with a...
1,26540,The DPR dropdown and actions are not visible o...
2,4320,Optimizer incorrectly removes unused local var...
3,14545,The insecure password warning does not appear ...
4,23198,The connection to the IRC server is not being ...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Mozilla crashes when installing a theme with a...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The DPR dropdown and actions are not visible o...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimizer incorrectly removes unused local var...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,The insecure password warning does not appear ...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,The connection to the IRC server is not being ...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 2319.84 seconds, 0.22 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.368237167596817, R=0.5544751882553101, F1=0.4425612986087799
Row 1, Bug ID 26540: BERTScore P=0.4033103883266449, R=0.5074692964553833, F1=0.4494338929653168
Row 2, Bug ID 4320: BERTScore P=0.35743218660354614, R=0.5008029341697693, F1=0.4171423017978668
Row 3, Bug ID 14545: BERTScore P=0.37008026242256165, R=0.6035779714584351, F1=0.4588310122489929
Row 4, Bug ID 23198: BERTScore P=0.371903657913208, R=0.4699929654598236, F1=0.4152340888977051
Row 5, Bug ID 5663: BERTScore P=0.38442954421043396, R=0.5396112203598022, F1=0.4489898979663849
Row 6, Bug ID 9750: BERTScore P=0.3237816393375397, R=0.6982433795928955, F1=0.4424126446247101
Row 7, Bug ID 18076: BERTScore P=0.38877737522125244, R=0.570315957069397, F1=0.4623657166957855
Row 8, Bug ID 6580: BERTScore P=0.29982465505599976, R=0.40582185983657837, F1=0.3448621928691864
Row 9, Bug ID 4252: BERTScore P=0.3621528446674347, R=0.5199841856956482, F1=0.42694

In [ ]:
output_file2 = "/content/drive/My Drive/mistral-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Theme installer crashes with malformed URL input.
1,26540,Dark theme hides device selection drop-down in...
2,4320,Optimizing unused local variables removes fiel...
3,14545,Password manager warning does not show for in ...
4,23198,Connection to IRC server keeps getting closed.


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Theme installer crashes with malformed URL input.,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,Dark theme hides device selection drop-down in...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimizing unused local variables removes fiel...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password manager warning does not show for in ...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to IRC server keeps getting closed.,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")


calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 946.87 seconds, 0.53 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.6651941537857056, R=0.5888252258300781, F1=0.6246842741966248
Row 1, Bug ID 26540: BERTScore P=0.6819074749946594, R=0.594749927520752, F1=0.6353536248207092
Row 2, Bug ID 4320: BERTScore P=0.705351710319519, R=0.6118288636207581, F1=0.6552700996398926
Row 3, Bug ID 14545: BERTScore P=0.7766286134719849, R=0.759429931640625, F1=0.7679329514503479
Row 4, Bug ID 23198: BERTScore P=0.6062939167022705, R=0.5284319519996643, F1=0.5646916031837463
Row 5, Bug ID 5663: BERTScore P=0.7017781138420105, R=0.6237610578536987, F1=0.6604737043380737
Row 6, Bug ID 9750: BERTScore P=0.6429570317268372, R=0.7502785921096802, F1=0.6924842596054077
Row 7, Bug ID 18076: BERTScore P=0.6304678916931152, R=0.5762873888015747, F1=0.6021613478660583
Row 8, Bug ID 6580: BERTScore P=0.33473774790763855, R=0.505001425743103, F1=0.4026084244251251
Row 9, Bug ID 4252: BERTScore P=0.6384190320968628, R=0.7141873240470886, F1=0.674181044101

In [ ]:
output_file2 = "/content/drive/My Drive/phi-3-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Bad URL in Install theme button causes Firefox...
1,26540,"Dark theme hides DPM/DPR indicators, misleadin..."
2,4320,Compiler optimizes away unreferenced local var...
3,14545,Incorrect fix causes password warning to not a...
4,23198,Connection to chat server closed unexpectedly....


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Bad URL in Install theme button causes Firefox...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,"Dark theme hides DPM/DPR indicators, misleadin...",[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Compiler optimizes away unreferenced local var...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Incorrect fix causes password warning to not a...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to chat server closed unexpectedly....,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")


calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 1804.78 seconds, 0.28 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.3886434733867645, R=0.5334871411323547, F1=0.449689656496048
Row 1, Bug ID 26540: BERTScore P=0.44810229539871216, R=0.4739187955856323, F1=0.46064913272857666
Row 2, Bug ID 4320: BERTScore P=0.5205540060997009, R=0.6604925394058228, F1=0.5822328329086304
Row 3, Bug ID 14545: BERTScore P=0.4233771562576294, R=0.6027629375457764, F1=0.4973903000354767
Row 4, Bug ID 23198: BERTScore P=0.3791535496711731, R=0.48468393087387085, F1=0.4254726767539978
Row 5, Bug ID 5663: BERTScore P=0.44634440541267395, R=0.5443994998931885, F1=0.4905196726322174
Row 6, Bug ID 9750: BERTScore P=0.38014695048332214, R=0.8092646598815918, F1=0.5172969102859497
Row 7, Bug ID 18076: BERTScore P=0.502694845199585, R=0.5613589882850647, F1=0.5304097533226013
Row 8, Bug ID 6580: BERTScore P=0.43876510858535767, R=0.508594274520874, F1=0.4711061716079712
Row 9, Bug ID 4252: BERTScore P=0.4044800400733948, R=0.5207766890525818, F1=0.45531

In [ ]:
output_file2 = "/content/drive/My Drive/gemma-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,"Sure, here is a summary in one sentence : The ..."
1,26540,"Sure, here is a summary in one sentence:\n\nTh..."
2,4320,"Sure, here's a summary in one sentence: The c..."
3,14545,Password manager disables insecure password wa...
4,23198,The connection to the server is being closed p...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,"Sure, here is a summary in one sentence : The ...",I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,"Sure, here is a summary in one sentence:\n\nTh...",[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,"Sure, here's a summary in one sentence: The c...","Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password manager disables insecure password wa...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,The connection to the server is being closed p...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 960.52 seconds, 0.52 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.5169313549995422, R=0.6015614867210388, F1=0.5560446381568909
Row 1, Bug ID 26540: BERTScore P=0.5508568286895752, R=0.5899878144264221, F1=0.5697512626647949
Row 2, Bug ID 4320: BERTScore P=0.5847030878067017, R=0.627683162689209, F1=0.6054312586784363
Row 3, Bug ID 14545: BERTScore P=0.773167610168457, R=0.6718673706054688, F1=0.7189668416976929
Row 4, Bug ID 23198: BERTScore P=0.6041220426559448, R=0.5276893377304077, F1=0.5633249282836914
Row 5, Bug ID 5663: BERTScore P=0.6344938278198242, R=0.6407777667045593, F1=0.6376202702522278
Row 6, Bug ID 9750: BERTScore P=0.5884346961975098, R=0.8350487351417542, F1=0.6903792023658752
Row 7, Bug ID 18076: BERTScore P=0.556158721446991, R=0.5587767362594604, F1=0.5574646592140198
Row 8, Bug ID 6580: BERTScore P=0.5319517850875854, R=0.5069152116775513, F1=0.5191318392753601
Row 9, Bug ID 4252: BERTScore P=0.5387119650840759, R=0.6544250249862671, F1=0.590957462787

In [ ]:
output_file2 = "/content/drive/My Drive/qwen-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,The user is unable to install a theme due to a...
1,26540,The issue is that the DPR dropdown and actions...
2,4320,The issue is that the compiler fails to recogn...
3,14545,The bug is that when the Password Manager is t...
4,23198,The user is unable to connect to the IRC serve...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,The user is unable to install a theme due to a...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The issue is that the DPR dropdown and actions...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,The issue is that the compiler fails to recogn...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,The bug is that when the Password Manager is t...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,The user is unable to connect to the IRC serve...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 1948.44 seconds, 0.26 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.4011242389678955, R=0.4997614622116089, F1=0.44504299759864807
Row 1, Bug ID 26540: BERTScore P=0.40616026520729065, R=0.5424405932426453, F1=0.46451106667518616
Row 2, Bug ID 4320: BERTScore P=0.45135432481765747, R=0.5678329467773438, F1=0.5029377341270447
Row 3, Bug ID 14545: BERTScore P=0.43202075362205505, R=0.6357580423355103, F1=0.5144523978233337
Row 4, Bug ID 23198: BERTScore P=0.39568182826042175, R=0.48870381712913513, F1=0.4373006820678711
Row 5, Bug ID 5663: BERTScore P=0.4621593654155731, R=0.5902332663536072, F1=0.518403172492981
Row 6, Bug ID 9750: BERTScore P=0.3671485185623169, R=0.6496140956878662, F1=0.46914559602737427
Row 7, Bug ID 18076: BERTScore P=0.34663718938827515, R=0.4806310534477234, F1=0.40278252959251404
Row 8, Bug ID 6580: BERTScore P=0.43504875898361206, R=0.4922323226928711, F1=0.46187734603881836
Row 9, Bug ID 4252: BERTScore P=0.38140806555747986, R=0.570173978805542, F1

In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,The bug occurs when trying to install a theme ...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,The DevEdition Dynamic Pixel Ratio (DPR) optio...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,The compiler does not generate a call to the c...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password Manager is not working as expected wh...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to IRC server is failing due to SOC...,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
output_file2 = "/content/drive/My Drive/deepseek-br-ft-eval-sumllama.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,The bug occurs when trying to install a theme ...
1,26540,The DevEdition Dynamic Pixel Ratio (DPR) optio...
2,4320,The compiler does not generate a call to the c...
3,14545,Password Manager is not working as expected wh...
4,23198,Connection to IRC server is failing due to SOC...


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 1970.58 seconds, 0.25 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.40829652547836304, R=0.5578619241714478, F1=0.47150254249572754
Row 1, Bug ID 26540: BERTScore P=0.44370269775390625, R=0.5906479358673096, F1=0.5067374110221863
Row 2, Bug ID 4320: BERTScore P=0.4458119571208954, R=0.6017167568206787, F1=0.5121626257896423
Row 3, Bug ID 14545: BERTScore P=0.38861584663391113, R=0.5708733797073364, F1=0.46243447065353394
Row 4, Bug ID 23198: BERTScore P=0.36312779784202576, R=0.5048575401306152, F1=0.4224214255809784
Row 5, Bug ID 5663: BERTScore P=0.4092124402523041, R=0.6168876886367798, F1=0.49203407764434814
Row 6, Bug ID 9750: BERTScore P=0.26748043298721313, R=0.5506165027618408, F1=0.36005303263664246
Row 7, Bug ID 18076: BERTScore P=0.44908711314201355, R=0.5691941976547241, F1=0.5020572543144226
Row 8, Bug ID 6580: BERTScore P=0.36753785610198975, R=0.5030727982521057, F1=0.42475542426109314
Row 9, Bug ID 4252: BERTScore P=0.4146226644515991, R=0.6790995597839355, F

In [ ]:
output_file2 = "/content/drive/My Drive/mistral-br-ft-eval-sumllama-with-code.csv"

df2 = pd.read_csv(output_file2)
df2.head()

,bug.id,summary
0,18027,Installing a theme with an invalid URL causes ...
1,26540,Dark theme makes it impossible to tell if 'Dev...
2,4320,Optimizing unused local variables removes a ca...
3,14545,Password manager off does not show in secure p...
4,23198,Connection to IRC server keeps getting closed.


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Installing a theme with an invalid URL causes ...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,Dark theme makes it impossible to tell if 'Dev...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimizing unused local variables removes a ca...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Password manager off does not show in secure p...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection to IRC server keeps getting closed.,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 912.37 seconds, 0.55 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.7360286116600037, R=0.7077356576919556, F1=0.7216050028800964
Row 1, Bug ID 26540: BERTScore P=0.5937188267707825, R=0.5764548778533936, F1=0.5849595069885254
Row 2, Bug ID 4320: BERTScore P=0.6887773275375366, R=0.6228135824203491, F1=0.6541367173194885
Row 3, Bug ID 14545: BERTScore P=0.7372227907180786, R=0.6816350221633911, F1=0.7083399891853333
Row 4, Bug ID 23198: BERTScore P=0.6062938570976257, R=0.5284318923950195, F1=0.5646914839744568
Row 5, Bug ID 5663: BERTScore P=0.6965120434761047, R=0.6074280738830566, F1=0.6489270329475403
Row 6, Bug ID 9750: BERTScore P=0.6549899578094482, R=0.8119599223136902, F1=0.7250766754150391
Row 7, Bug ID 18076: BERTScore P=0.6033623814582825, R=0.5892090797424316, F1=0.5962017774581909
Row 8, Bug ID 6580: BERTScore P=0.5961150527000427, R=0.4864937961101532, F1=0.5357545018196106
Row 9, Bug ID 4252: BERTScore P=0.6384835243225098, R=0.7068198919296265, F1=0.670916080

In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,ground_truth_summary
0,18027,Mozilla blocks and crashes when using invalid ...,I've the 2000052420 Mozilla version In Edit -...,Installing thme from a bad URL crahses Mozilla
1,26540,Dark theme makes DPR difficult to see/click in...,[Affected versions]: - Latest Nightly 53.0a1 -...,Show that DPR is not changeable in a better wa...
2,4320,Optimization bug in SingleNameReference affect...,"Using HEAD, interface I { int CONST = A.f...",[compiler] Unused locals initialisation is opt...
3,14545,Insecure password warning not showing with dis...,Created attachment 8830619 Test case I totall...,Insecure password field warning doesn't appear...
4,23198,Connection errors with IRC server using proxy.,*** Original post on bio 2251 at 2013-11-15 11...,"Accounts stuck in the ""Connecting..."" state"


In [ ]:
from bert_score import score


def evaluate_bert(ground_truth_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, ground_truth_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["ground_truth_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["ground_truth_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/16 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/8 [00:00<?, ?it/s]

done in 689.55 seconds, 0.73 sentences/sec
Row 0, Bug ID 18027: BERTScore P=0.65349280834198, R=0.6228258609771729, F1=0.6377909183502197
Row 1, Bug ID 26540: BERTScore P=0.6317371726036072, R=0.5656999349594116, F1=0.5968976616859436
Row 2, Bug ID 4320: BERTScore P=0.5865192413330078, R=0.562629222869873, F1=0.5743259191513062
Row 3, Bug ID 14545: BERTScore P=0.857665479183197, R=0.788204550743103, F1=0.8214693069458008
Row 4, Bug ID 23198: BERTScore P=0.5346270799636841, R=0.5257875323295593, F1=0.5301705002784729
Row 5, Bug ID 5663: BERTScore P=0.6976574063301086, R=0.5875331163406372, F1=0.6378771066665649
Row 6, Bug ID 9750: BERTScore P=0.7276206612586975, R=0.8739397525787354, F1=0.7940963506698608
Row 7, Bug ID 18076: BERTScore P=0.5588741302490234, R=0.5476723909378052, F1=0.5532165765762329
Row 8, Bug ID 6580: BERTScore P=0.621752142906189, R=0.48203739523887634, F1=0.5430523753166199
Row 9, Bug ID 4252: BERTScore P=0.5793131589889526, R=0.6067761778831482, F1=0.59272676706314